# CIRC Coordination Demonstration: Friday CT Slot Collision

**Supplementary Material for:** *Asimov's Laws for Clinical AI Agents: Protocol-Level Governance for Multi-Agent Coordination*

**Authors:** Bruce Changlong Xu, Ayush Chopra, Alexander Ryu

---

This notebook demonstrates the coordination failures described in Section 4 of the CIRC paper and quantifies how protocol-level coordination (L3) resolves them.

**Scenario:** A regional cancer network with limited Friday CT imaging slots. Multiple autonomous scheduling agents from different vendors independently optimize for their patients' pre-chemotherapy imaging. Without coordination (L1), agents converge on the same "optimal" slots, creating collisions and retry storms. With CIRC-L3 coordination, agents share utilization signals and respect dependency registries.

**Connection to REP:** This demonstration extends the Ripple Effect Protocol (Chopra et al., 2025) to healthcare-specific coordination failures, showing how aggregate resource signals and structured dependencies prevent the "digital stampede" in clinical settings.

In [ ]:
import random
import time
from dataclasses import dataclass, field
from enum import Enum
from typing import Optional
from collections import defaultdict
import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

# Set style for publication
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['figure.dpi'] = 150

# Reproducibility
random.seed(42)
np.random.seed(42)

## 1. Data Structures

We model the TCHP cohort from the paper: 180 patients receiving adjuvant TCHP for HER2-positive breast cancer, with 51% having concurrent type 2 diabetes requiring glucose optimization before contrast imaging.

In [ ]:
class AgentType(Enum):
    ONCOLOGY = "oncology"
    DIABETES = "diabetes"
    SCHEDULING = "scheduling"

class SlotStatus(Enum):
    AVAILABLE = "available"
    BOOKED = "booked"

@dataclass
class Patient:
    """TCHP cohort patient"""
    id: str
    has_diabetes: bool
    glucose_optimized: bool
    priority: int  # 1=urgent, 5=routine

@dataclass
class ImagingSlot:
    """Friday CT slot"""
    id: str
    time: str
    status: SlotStatus = SlotStatus.AVAILABLE
    booked_for: Optional[str] = None
    booked_by: Optional[str] = None

@dataclass
class BookingRequest:
    """Agent request for imaging slot"""
    agent_id: str
    patient_id: str
    preferred_slot: str
    priority: int
    timestamp: float
    prerequisites_met: bool = True

## 2. CIRC-L1: Isolated Agent Operation

> *"At Level 1, autonomous agents interact with clinical systems but not with each other. Each agent accesses the EHR, scheduling systems, and clinical decision support tools through standard interfaces, executing actions within its designated scope. The infrastructure treats each agent as an independent actor, unaware of other agents operating on the same patient or competing for the same resources."*

**Failure modes:**
- Resource collision (timestamp-based queuing)
- Retry storms when rejections provide no alternatives
- Cross-domain prerequisite violations (diabetes agent not consulted)

In [ ]:
class L1Environment:
    """CIRC-L1: No inter-agent coordination"""
    
    def __init__(self, num_slots: int = 20):
        self.slots = {}
        for i in range(num_slots):
            hour = 13 + (i // 4)  # 1pm - 6pm ("optimal" afternoon window)
            minute = (i % 4) * 15
            slot_id = f"FRI-{hour:02d}{minute:02d}"
            self.slots[slot_id] = ImagingSlot(id=slot_id, time=f"{hour:02d}:{minute:02d}")
        
        self.collision_log = []
        self.retry_log = []
        self.booking_log = []
    
    def get_available_slots(self):
        return [s for s in self.slots.values() if s.status == SlotStatus.AVAILABLE]
    
    def submit_booking(self, request: BookingRequest):
        slot = self.slots.get(request.preferred_slot)
        if slot is None:
            return False, "INVALID_SLOT"
        
        if slot.status == SlotStatus.BOOKED:
            # L1: "Agents receive rejection notices without information about why
            # or what alternatives might succeed"
            self.collision_log.append({
                "agent": request.agent_id,
                "patient": request.patient_id,
                "slot": request.preferred_slot
            })
            return False, "SLOT_UNAVAILABLE"
        
        slot.status = SlotStatus.BOOKED
        slot.booked_for = request.patient_id
        slot.booked_by = request.agent_id
        self.booking_log.append({"patient": request.patient_id, "slot": request.preferred_slot})
        return True, "BOOKED"


class L1SchedulingAgent:
    """L1 Scheduling Agent: Optimizes independently"""
    
    def __init__(self, agent_id: str, vendor: str):
        self.id = agent_id
        self.vendor = vendor
        self.patients = []
    
    def add_patient(self, patient: Patient):
        self.patients.append(patient)
    
    def select_optimal_slot(self, available_slots, patient):
        """All agents converge on same 'optimal' afternoon slots"""
        if not available_slots:
            return None
        # Bias toward early afternoon (the "optimal" strategy)
        available_slots.sort(key=lambda s: s.id)
        idx = min(patient.priority - 1, len(available_slots) - 1)
        return available_slots[idx].id

## 3. CIRC-L3: Universal Coordination Protocol

> *"At Level 3, a shared coordination protocol enables agents from different vendors and institutions to exchange structured messages, declare capabilities and constraints, and negotiate over shared patient state regardless of underlying implementation."*

**Key primitives:**
- Dependency registry (agents register interests)
- Utilization signals (aggregate resource awareness)
- Structured notifications and arbitration

In [ ]:
class L3Environment:
    """CIRC-L3: Universal coordination across vendor ecosystems"""
    
    def __init__(self, num_slots: int = 20):
        self.slots = {}
        for i in range(num_slots):
            hour = 13 + (i // 4)
            minute = (i % 4) * 15
            slot_id = f"FRI-{hour:02d}{minute:02d}"
            self.slots[slot_id] = ImagingSlot(id=slot_id, time=f"{hour:02d}:{minute:02d}")
        
        # L3 coordination primitives
        self.dependency_registry = defaultdict(list)
        self.collision_log = []
        self.booking_log = []
        self.arbitration_log = []
        self.coordination_messages = []
    
    def register_interest(self, agent_id: str, patient_id: str):
        if agent_id not in self.dependency_registry[patient_id]:
            self.dependency_registry[patient_id].append(agent_id)
    
    def query_utilization(self):
        """L3: Agents can query aggregate resource state"""
        booked = sum(1 for s in self.slots.values() if s.status == SlotStatus.BOOKED)
        return {
            "total": len(self.slots),
            "available": len(self.slots) - booked,
            "utilization": booked / len(self.slots)
        }
    
    def get_available_slots(self):
        return [s for s in self.slots.values() if s.status == SlotStatus.AVAILABLE]
    
    def submit_booking(self, request: BookingRequest):
        slot = self.slots.get(request.preferred_slot)
        if slot is None:
            return False, "INVALID_SLOT"
        
        if slot.status == SlotStatus.BOOKED:
            util = self.query_utilization()
            self.collision_log.append({"patient": request.patient_id, "slot": request.preferred_slot})
            return False, f"SLOT_UNAVAILABLE:available={util['available']}"
        
        # L3: Check with dependency registry
        interested = self.dependency_registry.get(request.patient_id, [])
        
        # Check prerequisites
        if not request.prerequisites_met:
            self.arbitration_log.append({
                "patient": request.patient_id,
                "reason": "Prerequisites not met (glucose not optimized)"
            })
            return False, "BLOCKED:prerequisites"
        
        slot.status = SlotStatus.BOOKED
        slot.booked_for = request.patient_id
        slot.booked_by = request.agent_id
        self.booking_log.append({"patient": request.patient_id, "slot": request.preferred_slot})
        self.coordination_messages.append({"type": "booking", "coordinated_with": interested})
        return True, "BOOKED"


class L3SchedulingAgent:
    """L3 Scheduling Agent: Coordinates via protocol"""
    
    def __init__(self, agent_id: str, vendor: str):
        self.id = agent_id
        self.vendor = vendor
        self.patients = []
    
    def add_patient(self, patient: Patient):
        self.patients.append(patient)
    
    def select_slot_with_utilization(self, env, patient):
        """L3: Use utilization signal to distribute load"""
        available = env.get_available_slots()
        if not available:
            return None
        # Distribute more evenly based on utilization awareness
        available.sort(key=lambda s: s.id)
        # Offset by utilization to spread load
        util = env.query_utilization()
        offset = int(util['utilization'] * len(available))
        idx = (patient.priority - 1 + offset) % len(available)
        return available[idx].id


class L3DiabetesAgent:
    """L3 Diabetes Agent: Participates in dependency registry"""
    
    def __init__(self, agent_id: str):
        self.id = agent_id
        self.patients = []
    
    def add_patient(self, patient: Patient):
        if patient.has_diabetes:
            self.patients.append(patient)

## 4. Simulation Runner

In [ ]:
def create_tchp_cohort(n_patients: int = 40, diabetes_rate: float = 0.51):
    """Create TCHP cohort (51% diabetic per paper statistics)"""
    patients = []
    for i in range(n_patients):
        has_diabetes = random.random() < diabetes_rate
        patients.append(Patient(
            id=f"TCHP-{i:03d}",
            has_diabetes=has_diabetes,
            glucose_optimized=random.random() > 0.3 if has_diabetes else True,
            priority=random.choices([1, 2, 3, 4, 5], weights=[5, 15, 30, 30, 20])[0]
        ))
    return patients


def run_l1_simulation(patients, n_slots=20, n_agents=3):
    """L1: Concurrent agent execution with collision"""
    env = L1Environment(num_slots=n_slots)
    agents = [L1SchedulingAgent(f"SCHED-{i}", f"Vendor-{chr(65+i)}") for i in range(n_agents)]
    
    for i, patient in enumerate(patients):
        agents[i % n_agents].add_patient(patient)
    
    # All agents select slots BEFORE any booking (concurrent execution)
    all_requests = []
    for agent in agents:
        available = env.get_available_slots()
        for patient in agent.patients:
            slot_id = agent.select_optimal_slot(available, patient)
            if slot_id:
                all_requests.append(BookingRequest(
                    agent_id=agent.id,
                    patient_id=patient.id,
                    preferred_slot=slot_id,
                    priority=patient.priority,
                    timestamp=time.time() + random.random() * 0.01
                ))
    
    all_requests.sort(key=lambda r: r.timestamp)
    
    results = {"booked": 0, "failed": 0, "retries": 0}
    for request in all_requests:
        success, _ = env.submit_booking(request)
        if success:
            results["booked"] += 1
        else:
            results["failed"] += 1
            # L1: Blind retry
            for _ in range(3):
                available = env.get_available_slots()
                if not available:
                    break
                results["retries"] += 1
                env.retry_log.append({"agent": request.agent_id})
                retry_req = BookingRequest(
                    agent_id=request.agent_id,
                    patient_id=request.patient_id,
                    preferred_slot=random.choice(available).id,
                    priority=request.priority,
                    timestamp=time.time()
                )
                if env.submit_booking(retry_req)[0]:
                    results["booked"] += 1
                    break
    
    return {
        "collisions": len(env.collision_log),
        "retries": len(env.retry_log),
        "booked": results["booked"],
        "failed": results["failed"]
    }


def run_l3_simulation(patients, n_slots=20, n_agents=3):
    """L3: Coordinated execution with dependency registry"""
    env = L3Environment(num_slots=n_slots)
    sched_agents = [L3SchedulingAgent(f"SCHED-{i}", f"Vendor-{chr(65+i)}") for i in range(n_agents)]
    diabetes_agent = L3DiabetesAgent("DIABETES-001")
    
    for i, patient in enumerate(patients):
        sched_agents[i % n_agents].add_patient(patient)
        diabetes_agent.add_patient(patient)
    
    # Register interests
    for agent in sched_agents:
        for patient in agent.patients:
            env.register_interest(agent.id, patient.id)
            if patient.has_diabetes:
                env.register_interest(diabetes_agent.id, patient.id)
    
    results = {"booked": 0, "blocked": 0, "retries": 0}
    
    for agent in sched_agents:
        for patient in agent.patients:
            slot_id = agent.select_slot_with_utilization(env, patient)
            if not slot_id:
                continue
            
            prereq_met = not patient.has_diabetes or patient.glucose_optimized
            
            request = BookingRequest(
                agent_id=agent.id,
                patient_id=patient.id,
                preferred_slot=slot_id,
                priority=patient.priority,
                timestamp=time.time(),
                prerequisites_met=prereq_met
            )
            
            success, msg = env.submit_booking(request)
            if success:
                results["booked"] += 1
            elif "BLOCKED" in msg:
                results["blocked"] += 1
            else:
                results["retries"] += 1
    
    return {
        "collisions": len(env.collision_log),
        "retries": results["retries"],
        "booked": results["booked"],
        "blocked": results["blocked"]
    }

## 5. Run Comparative Simulation

In [ ]:
# Parameters
N_RUNS = 50
N_PATIENTS = 40
N_SLOTS = 20  # Limited "optimal" slots creates contention

l1_results = {"collisions": [], "retries": [], "booked": []}
l3_results = {"collisions": [], "retries": [], "booked": [], "blocked": []}

for run in range(N_RUNS):
    patients = create_tchp_cohort(N_PATIENTS)
    
    l1 = run_l1_simulation(patients.copy(), N_SLOTS)
    l1_results["collisions"].append(l1["collisions"])
    l1_results["retries"].append(l1["retries"])
    l1_results["booked"].append(l1["booked"])
    
    l3 = run_l3_simulation(patients.copy(), N_SLOTS)
    l3_results["collisions"].append(l3["collisions"])
    l3_results["retries"].append(l3["retries"])
    l3_results["booked"].append(l3["booked"])
    l3_results["blocked"].append(l3["blocked"])

# Summary statistics
summary = {
    "L1": {k: (np.mean(v), np.std(v)) for k, v in l1_results.items()},
    "L3": {k: (np.mean(v), np.std(v)) for k, v in l3_results.items()}
}

print(f"Simulation: {N_RUNS} runs, {N_PATIENTS} patients, {N_SLOTS} slots")
print("\nL1 (Isolated):")
print(f"  Collisions: {summary['L1']['collisions'][0]:.1f} ± {summary['L1']['collisions'][1]:.1f}")
print(f"  Retries: {summary['L1']['retries'][0]:.1f} ± {summary['L1']['retries'][1]:.1f}")
print(f"  Booked: {summary['L1']['booked'][0]:.1f} ± {summary['L1']['booked'][1]:.1f}")

print("\nL3 (Coordinated):")
print(f"  Collisions: {summary['L3']['collisions'][0]:.1f} ± {summary['L3']['collisions'][1]:.1f}")
print(f"  Retries: {summary['L3']['retries'][0]:.1f} ± {summary['L3']['retries'][1]:.1f}")
print(f"  Booked: {summary['L3']['booked'][0]:.1f} ± {summary['L3']['booked'][1]:.1f}")
print(f"  Blocked (prerequisite): {summary['L3']['blocked'][0]:.1f} ± {summary['L3']['blocked'][1]:.1f}")

## 6. Publication Figure

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# Color scheme
L1_COLOR = '#E74C3C'  # Red for failures
L3_COLOR = '#27AE60'  # Green for coordination
BLOCKED_COLOR = '#F39C12'  # Orange for caught violations

# Panel A: Collisions
ax = axes[0]
positions = [0, 1]
data = [l1_results["collisions"], l3_results["collisions"]]
bp = ax.boxplot(data, positions=positions, widths=0.6, patch_artist=True)
bp['boxes'][0].set_facecolor(L1_COLOR)
bp['boxes'][1].set_facecolor(L3_COLOR)
for box in bp['boxes']:
    box.set_alpha(0.7)
ax.set_xticks(positions)
ax.set_xticklabels(['L1\n(Isolated)', 'L3\n(Coordinated)'])
ax.set_ylabel('Slot Collisions per Cycle')
ax.set_title('A. Resource Collisions', fontweight='bold')
ax.set_ylim(bottom=-1)

# Add improvement annotation
l1_mean = np.mean(l1_results["collisions"])
l3_mean = np.mean(l3_results["collisions"])
if l1_mean > 0:
    reduction = (l1_mean - l3_mean) / l1_mean * 100
    ax.annotate(f'{reduction:.0f}% reduction', xy=(0.5, l1_mean/2), 
                fontsize=10, ha='center', fontweight='bold', color='#2C3E50')

# Panel B: Retry Storms
ax = axes[1]
data = [l1_results["retries"], l3_results["retries"]]
bp = ax.boxplot(data, positions=positions, widths=0.6, patch_artist=True)
bp['boxes'][0].set_facecolor(L1_COLOR)
bp['boxes'][1].set_facecolor(L3_COLOR)
for box in bp['boxes']:
    box.set_alpha(0.7)
ax.set_xticks(positions)
ax.set_xticklabels(['L1\n(Isolated)', 'L3\n(Coordinated)'])
ax.set_ylabel('Retry Attempts per Cycle')
ax.set_title('B. Retry Storms', fontweight='bold')
ax.set_ylim(bottom=-1)

# Panel C: Outcomes breakdown
ax = axes[2]
width = 0.35
x = np.array([0, 1])

# L1: booked (some unsafe)
l1_booked = np.mean(l1_results["booked"])
# L3: booked (safe) + blocked (caught)
l3_booked = np.mean(l3_results["booked"])
l3_blocked = np.mean(l3_results["blocked"])

bars1 = ax.bar(x - width/2, [l1_booked, l3_booked], width, 
               label='Successfully Booked', color=L3_COLOR, alpha=0.7)
bars2 = ax.bar(x[1] + width/2, [l3_blocked], width,
               label='Blocked (prerequisite)', color=BLOCKED_COLOR, alpha=0.7)

ax.set_xticks(x)
ax.set_xticklabels(['L1\n(Isolated)', 'L3\n(Coordinated)'])
ax.set_ylabel('Patients per Cycle')
ax.set_title('C. Booking Outcomes', fontweight='bold')
ax.legend(loc='upper right', fontsize=9)

# Add annotation for blocked
ax.annotate(f'{l3_blocked:.1f} violations\ncaught', 
            xy=(1 + width/2, l3_blocked + 0.5),
            ha='center', fontsize=9, color='#D35400')

plt.tight_layout()
plt.savefig('circ_coordination_demo.png', dpi=300, bbox_inches='tight', 
            facecolor='white', edgecolor='none')
plt.savefig('circ_coordination_demo.pdf', bbox_inches='tight')
plt.show()

print("\nFigure saved as circ_coordination_demo.png and .pdf")

## 7. Figure Caption

**Figure S1. CIRC coordination demonstration: Friday CT slot collision scenario.**

Simulation of 40 TCHP patients requiring pre-chemotherapy imaging, with 3 scheduling agents from different vendors competing for 20 "optimal" Friday afternoon CT slots. (A) Resource collisions under L1 (isolated operation) versus L3 (coordinated protocol). At L1, agents independently converge on the same slots before any bookings are confirmed, generating ~30 collisions per cycle. L3's utilization signals and dependency registry eliminate collisions. (B) Retry storms caused by L1 rejections that provide no alternative guidance. L3's informed rejections include availability data, preventing retry cascades. (C) Booking outcomes showing L3's detection of prerequisite violations (diabetic patients without glucose optimization) that would proceed to unsafe imaging under L1. Results from 50 simulation runs. Parameters: 51% diabetes rate, 30% of diabetic patients not glucose-optimized. Connects to Chopra et al.'s Ripple Effect Protocol (REP) demonstrating healthcare-specific coordination failures.

## 8. Export Data for Reproducibility

In [ ]:
output = {
    "scenario": "Friday CT Slot Collision",
    "paper": "Asimov's Laws for Clinical AI Agents (Xu, Chopra, Ryu 2025)",
    "connection": "Extends REP (Chopra et al. 2025) to clinical coordination",
    "parameters": {
        "n_runs": N_RUNS,
        "n_patients": N_PATIENTS,
        "n_slots": N_SLOTS,
        "n_agents": 3,
        "diabetes_rate": 0.51,
        "glucose_optimization_rate": 0.70
    },
    "results": {
        "L1": {
            "collisions_mean": float(np.mean(l1_results["collisions"])),
            "collisions_std": float(np.std(l1_results["collisions"])),
            "retries_mean": float(np.mean(l1_results["retries"])),
            "retries_std": float(np.std(l1_results["retries"])),
            "booked_mean": float(np.mean(l1_results["booked"])),
            "booked_std": float(np.std(l1_results["booked"]))
        },
        "L3": {
            "collisions_mean": float(np.mean(l3_results["collisions"])),
            "collisions_std": float(np.std(l3_results["collisions"])),
            "retries_mean": float(np.mean(l3_results["retries"])),
            "retries_std": float(np.std(l3_results["retries"])),
            "booked_mean": float(np.mean(l3_results["booked"])),
            "booked_std": float(np.std(l3_results["booked"])),
            "blocked_mean": float(np.mean(l3_results["blocked"])),
            "blocked_std": float(np.std(l3_results["blocked"]))
        }
    },
    "clinical_interpretation": {
        "L1_failures": [
            "Retry storms consume system resources",
            "Prerequisite violations cause hypoglycemia (diabetic patients imaged without glucose optimization)",
            "No information about alternatives on rejection"
        ],
        "L3_benefits": [
            "Utilization signals distribute load across time slots",
            "Dependency registry enables cross-domain coordination",
            "Unsafe bookings blocked before execution",
            "Informed rejections prevent retry cascades"
        ]
    }
}

with open('circ_simulation_results.json', 'w') as f:
    json.dump(output, f, indent=2)

print("Results exported to circ_simulation_results.json")
print("\nKey findings:")
print(f"  - Collision reduction: {(np.mean(l1_results['collisions']) - np.mean(l3_results['collisions'])) / np.mean(l1_results['collisions']) * 100:.0f}%")
print(f"  - Retry reduction: {(np.mean(l1_results['retries']) - np.mean(l3_results['retries'])) / max(np.mean(l1_results['retries']), 1) * 100:.0f}%")
print(f"  - Prerequisite violations caught: {np.mean(l3_results['blocked']):.1f} per cycle")